In [ ]:
import os
import json
import numpy as np
import pandas as pd
import scipy.spatial
from scipy.spatial.transform import Rotation as R

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

object_dir_path = "/home/zimmermc/projects/RGBD_Fusion/data/pose_estimation/"
object_dirs = [name for name in os.listdir(object_dir_path) if os.path.isdir(os.path.join(object_dir_path, name))]
object_name = "wd_40"

print(object_dirs)
print(object_name)

assert(object_name in object_dirs)
object_dir = os.path.join(object_dir_path, object_name)
obj_files = sorted(os.listdir(object_dir))

# Load Images

In [ ]:
import PIL
rgb_files = []
depth_files = []
for fn in obj_files:
    if fn.startswith("rgb_") and fn.endswith(".png"):
        file_path = os.path.join(object_dir, fn)
        image = PIL.Image.open(file_path)
        rgb_files.append(image)
        
    elif fn.startswith("depth_") and fn.endswith(".png"):
        file_path = os.path.join(object_dir, fn)
        image = PIL.Image.open(file_path)
        depth_files.append(image)

assert len(rgb_files) == len(depth_files)
depth_scaling = 0.000125
print("images loaded.")

# Load Masks

In [ ]:
mask_dir_path = "/home/argusm/lang/flow_control/pose_estimation/"
mask_dir = os.path.join(mask_dir_path, object_name)
mask_fn = os.path.join(mask_dir, "mask.npz")
masks = np.load(mask_fn)["masks"]

In [ ]:
# convert to arrays
rgb_arr = np.array([np.array(x) for x in rgb_files])
depth_arr = np.array([np.array(x)*depth_scaling for x in depth_files])

# get a demo dict with one entry
def get_demo_dict(i):
    demo_dict = dict(rgb=rgb_arr[i:i+1],
                     depth=depth_arr[i:i+1],
                     mask=masks[i:i+1],
                     keep=[1],
                     state=np.ones((1,4)))
    
    return demo_dict

def get_live_arg(i):
    live_rgb = rgb_arr[i]
    ee_pos = None
    live_depth = depth_arr[i]
    return live_rgb, ee_pos, live_depth

In [ ]:
test_rgb, _, live_depth  = get_live_arg(0)
print(live_depth)

In [ ]:
save_path = "/home/argusm/CLUSTER/robot_recordings/pose_estimation"
save_dir = os.path.join(save_path,object_name)
dm_fn = os.path.join(save_dir,"dm.npz")
print(dm_fn)
assert(os.path.isfile(dm_fn))
dm_obj = np.load(dm_fn)

T_wt = dm_obj["T_world2tcp"]
T_mc = dm_obj["T_marker2cam"]
def get_transform(i,j):
    return T_mc[i] @ np.linalg.inv(T_mc[j])

assert len(rgb_files) == len(T_mc)
num_images = len(rgb_files)

# get sorted upper triangular values
dist_triu = np.triu_indices(num_images, 1)

dm_rot = dm_obj["rot_marker"]
# get distance ids and values
dist_rot_triu_values = dm_rot[dist_triu]
dist_rot_order = np.argsort(dist_rot_triu_values)
dist_ids = np.array(dist_triu).T[dist_rot_order]
dist_rot_values = dist_rot_triu_values[dist_rot_order]

print(dist_ids.shape)
print(dist_rot_values)

In [ ]:
from gym_grasping.flow_control.servoing_fitting import solve_transform
from gym_grasping.flow_control.servoing_module import ServoingModule

# this needs to have sample data to see sizes
default_demo = get_demo_dict(0)
#camera_calibration = dict(width=640, height=480, 
#                          ppx=128.0, ppy=128.0,
#                          fx=322.3224792221095, fy=322.3224792221095)
camera_calibration = dict(width=640, height=480,
                          ppx=315.20367431640625, ppy=245.70614624023438,
                          fx=617.8902587890625, fy=617.8903198242188)


class ServoingModuleFGR(ServoingModule):
    def get_transform_pc(self, live_rgb, ee_pos, live_depth):
        """
        get a transformation from a pointcloud using FGR.
        """
        assert live_rgb.shape == self.base_image_rgb.shape
        # 1. compute transformation
        # for compatibility with notebook.
        demo_rgb = self.base_image_rgb
        demo_depth = self.base_image_depth
        end_points = np.array(np.where(self.base_mask)).T

        start_pc = self.generate_pointcloud2(live_rgb, live_depth)
        end_pc = self.generate_pointcloud(demo_rgb, demo_depth, end_points)

        # transform into TCP coordinates
        T_tcp_cam = np.array([
            [0.99987185, -0.00306941, -0.01571176, 0.00169436],
            [-0.00515523, 0.86743151, -0.49752989, 0.11860651],
            [0.015156, 0.49754713, 0.86730453, -0.18967231],
            [0., 0., 0., 1.]])

        start_pc[:, 0:4] = (T_tcp_cam @ start_pc[:, 0:4].T).T
        end_pc[:, 0:4] = (T_tcp_cam @ end_pc[:, 0:4].T).T

        reg_res = self.reg_module.register(start_pc, end_pc)
        T_tp_t = reg_res.transformation
        guess = T_tp_t
        return guess

sm = ServoingModuleFGR(default_demo, camera_calibration=camera_calibration)

def estimate_transform(i, j):
    demo_dict = get_demo_dict(i)
    live_arg = get_live_arg(j)
    sm.set_demo(demo_dict)
    T_guess = sm.get_transform_pc(*live_arg)
    return T_guess

In [ ]:
from tqdm import tqdm
dist_stop = 250
dist_subsample = 1
filename = os.path.join(save_dir, "gt.npz")
print(filename)

plot_ids = []
rec = {}
for ids in tqdm(dist_ids[:dist_stop:dist_subsample]):
    i, j = ids
    plot_ids.append(ids)
    T_gt = get_transform(i, j)
    if not np.all(np.isfinite(T_gt)):
        continue
    rec['{}_{}'.format(i,j)] = T_gt
np.savez(filename,**rec)

# Run Pose Esimtation Algorithm (takes time)

In [ ]:
method_name = sm.method_name
print(method_name)
filename = os.path.join(save_dir, "{}.npz".format(method_name))
rec = {}
for ids in tqdm(dist_ids[:dist_stop:dist_subsample]):
    i, j = ids    
    T_gt = get_transform(i, j)
    if not np.all(np.isfinite(T_gt)):
        continue
    T_guess = estimate_transform(i, j)
    rec['{}_{}'.format(i,j)] = T_guess
np.savez(filename,**rec)
print("done.")

# Load Results

In [ ]:
mia_dir = "/misc/lmbraid19/zhouh/eval_wd40"
filenames = os.listdir(mia_dir)
algo_names = ("sift_mask", "deeptam")
algorithms = {}
for algo in algo_names:
    algorithms[algo] = os.path.join(mia_dir, '{}.npz'.format(algo))
algo_names = ("IRR_PWC", "FlowNet2", "FGR")
for algo in algo_names:
    algorithms[algo] = os.path.join(save_dir, '{}.npz'.format(algo))
# add zero prediction, so that we can use the same filtering code
algorithms["zero"] = None

results = {}
for algo, filename in algorithms.items():
    if filename is not None:
        file_obj = np.load(filename, allow_pickle=True)
        file_format = "dict"
        if 'arr_0' in file_obj:
            result = file_obj['arr_0']
            file_format = "array"
        else:
            result = file_obj
            file_format = 'dict'
    
    pos_errs = []
    rot_errs = []
    pos_mags = []
    rot_mags = []
    
    for ids in plot_ids:
        i,j = ids
        T_gt = get_transform(i, j)
        assert(np.all(np.isfinite(T_gt)))
        
        if algo == "zero":
            T_guess = np.eye(4)
        elif file_format == "array":
            T_guess = result[i+50*j]
        elif file_format == 'dict':
            T_guess = result['{}_{}'.format(i, j)]
        else:
            raise ValueError
        if T_guess is None:
            T_guess = np.eye(4)
            
        diff_pos = np.linalg.norm(T_gt[:3, 3]-T_guess[:3, 3], 2)
        R_gt = T_gt[:3, :3]
        R_ge = T_guess[:3, :3]
        diff_rot = R.from_matrix(R_gt @ np.linalg.inv(R_ge)).magnitude()
        
        if diff_pos > .25:
            diff_pos = .25
            
        pos_errs.append(diff_pos)
        rot_errs.append(diff_rot)
        pos_mags.append(np.linalg.norm(T_guess[:3,3]))
        rot_mags.append(R.from_matrix(R_ge).magnitude())
        
    results[algo] = dict(pos=pos_errs, rot=rot_errs,
                         pos_mag=pos_mags, rot_mag=rot_mags)
    
    print("done:", algo)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,figsize=(16,12))
algo = "IRR_PWC"
score = np.argsort(results[algo]["rot"])/len(results[algo]["rot"])
score = np.array(results[algo]["rot"])
im = sns.scatterplot(results["zero"]["rot"], results["zero"]["pos"], size=score, hue=score, label="error")
ax.set_xlabel("rot [rad]")
ax.set_ylabel("pos [m]")
#fig.colorbar(im, ax=ax)
plt.show()

In [ ]:
method_name = dict(FlowNet2="FlowNet2",
                   deeptam="DeepTAM", 
                   sift_mask="SIFT",
                   zero="Zero", IRR_PWC="IRR_PWC",FGR="FGR") 

fig, ax = plt.subplots(1)
ax.set_xlabel("increasing |R|")
ax.set_ylabel("error (rad)")
for k in method_name:
    v = results[k]
    ax.plot(v["rot"], label=method_name[k])
ax.legend()

Xs = results["zero"]["rot"]
fig, ax = plt.subplots(1)
ax.set_xlabel("increasing |R|")
ax.set_ylabel("error (m)")
for k in method_name:
    v = results[k]
    ax.plot(v["pos"], label=method_name[k])

ax.legend()

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# rotation and position masking thresholds
r_mask_t = np.pi/4
p_mask_t = .25

print("Number of samples:", len(plot_ids))
Xs = np.linspace(0, 1, len(plot_ids))

linew = 3.5
with sns.axes_style("ticks"):
    print("\nRotation: (AUC-higher is better)")
    fig, ax = plt.subplots(1, figsize=(4, 3))
    for algo in method_name:
        # find outliers to mask
        r_mask = np.array(results[algo]["rot_mag"]) > r_mask_t
        p_mask = np.array(results[algo]["pos_mag"]) > p_mask_t
        cur_mask = np.logical_or(r_mask, p_mask)
        # mask outliers
        data = np.array(results[algo]["rot"])
        data[cur_mask] = np.array(results["zero"]["rot"])[cur_mask]
        print(algo.ljust(10), 1-data.mean())
        # plot
        ax.plot(np.sort(data), Xs, label=method_name[algo], linewidth=linew)
    ax.set_ylabel("")
    ax.set_xlabel("error (rad)", size=18)
    ax.set_xlim(0,.8)
    #ax.legend(loc="lower right")    
    plt.savefig("/home/argusm/lang/flow_control/relative_orientation_error.pdf", bbox_inches="tight")

    print("\nTranslation:")
    fig, ax = plt.subplots(1, figsize=(4, 3))
    for algo in method_name:
        r_mask = np.array(results[algo]["rot_mag"]) > r_mask_t
        p_mask = np.array(results[algo]["pos_mag"]) > p_mask_t
        cur_mask = np.logical_or(r_mask, p_mask)
        data = np.array(results[algo]["pos"])
        data[cur_mask] = np.array(results["zero"]["pos"])[cur_mask]
        print(algo.ljust(10), 1-data.mean())
        ax.plot(np.sort(data)*1000, Xs, label=method_name[algo], linewidth=linew)
    ax.set_xlim(0, 250)
    ax.set_xlabel("error (mm)", size=18)
    ax.set_ylabel("")
    ax.legend(loc="lower right")
    plt.savefig("/home/argusm/lang/flow_control/relative_translation_error.pdf", bbox_inches="tight")

# Debugging Outlier Inspection